# CIS 5450 Homework 5: Deep Learning with Pytorch

### Due Tuesday, July 22, 2025 11:59 PM EST

<ins>Total Points</ins>: **100 points** (= 84 autograded + 16 manually graded)

Welcome to CIS 5450 Homework 5!

In this homework, we will learn more about the 'new electricity' - Deep Learning (I didn't coin this, Andrew Ng did) ! There are many cool frameworks for building deep learning models - PyTorch, Tensorflow, Theano, MxNet. Since you will be working with Big Data in this course, you need a framework that scales well. Almost all of these have a multi-gpu support built in. In this assignment, we will be building neural networks in PyTorch to solve an interesting problem.

Deep learning or neural network architectures have been used to solve a multitude of problems in various different fields like vision, natural language processing. So let's take a 'deep' dive into it.


![alt text](http://4.bp.blogspot.com/-sLgBLS3bJO8/U__Kx6PnkRI/AAAAAAAAAWM/Gz3cL5jiwTo/s320/godeeper.jpg)





## Why deep learning?


*   It's coooool
*   Everyone's talking about it. It's the foundation of almost all the 'AI' we see in the news today
*    Deep learning unlocks the treasure trove of unstructured big data for those with the imagination to use it
*   Deep learning models have great representational power and are 'universal approximators'


### Deep Learning Applications:


Deep learning is taking the world by storm.

Deep learning has applications across so many industries, which is why experts think that this technology is the future of almost everything. breakthrough LLMs such as ChatGPT, Claude and Llama are deep learning models. There are truly deep learning technologies such as Video generation models, Self Driving cars, and various budding ideas that are transforming the way we live. Here are some cool applications of deep learning -

Here's a neural network detecting anomalies in Chest Xrays :




![alt text](https://imgur.com/HKPzrzP.jpg)

Most humans can't tell that this is a case of Pleural Effusion {sounds like medical jargon to engineers like us} but this Neural Network model can detect it very well!

Mask RCNNs in action for detecting objects on the road aiding a self driving vehicle's driving:
<div>
<img src="https://miro.medium.com/max/3864/1*O4wsvh0CHWjP6IrK9HI_Zg.png", width="800"/>
</div>

Pretty cool, right? While transformers are exploding on the scene today, we will be applying CNNs, another powerful architecture, to solve a cool image classification problem.


# Libraries and Setup Jargon (Total: 1 point)
Run the following cells to set up the notebook.


Please make sure you enter your **8 digit Penn ID** in the  student ID field below.

In [ ]:
%%capture
!pip install -i https://test.pypi.org/simple/ penn-grader==0.5.0
from penngrader.grader import *

#PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO
#TO ASSIGN POINTS TO YOU IN OUR BACKEND

STUDENT_ID =       # YOUR PENN-ID GOES HERE AS AN INTEGER

In [ ]:
%%writefile config.yaml
grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Writing config.yaml


In [ ]:
grader = PennGrader('config.yaml', 'cis5450_su25_HW5', STUDENT_ID, STUDENT_ID)

## Import the required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from sklearn.model_selection import train_test_split
import torch
import torchvision
from torchvision import transforms, utils
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from collections import Counter
from PIL import Image
from skimage import io, transform
import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from google.colab import drive
from dill.source import getsource


## Set up GPU capabilities (1 point)

The cell below sets up a CUDA device to use with torch, if available to you.

**Remember to turn on the GPU for runtime in Colab**
*   Go to Runtime -> Change runtime type --> T4-GPU (Usually this is chosen by default)

In Colab, as a free-tier user you only have access to T4-GPU. There are dynamic usage limits on colab which is ~ 2 hours of GPU access everyday. Refer to the notes below for instructions on how to work with this.

Note that you can create a PyTorch tensor `T` that resides on the GPU using the command `T = torch.Tensor(..., device=cuda)`.
You can also copy existing tensors to the GPU with the command `T = T.to(device)` (make sure to overwrite `T` with the result of this function call). If you do not have a GPU, these commands will still work.

**NOTE 1**: The grader cell below is to check that you are using the GPU, since it is beneficial in later sections that require parallel computation on arrays (i.e., Section 2).

**NOTE 2**: It is possible that your Google Colab GPU credits may run out. In that case here are a couple of simple options to get past this:

* Continue the assignment using CPU. This will be a little slower but will still work. The autograder takes points from your best submission for each cell, so as long as you get the below points once, you will get these points. **Note that Colab selects a GPU runtime by-default when you load the notebook.**

* Use your personal email account. Your personal email account will have the same number of initial credits, and you can work from there by simply sharing this assignment with your personal id. Some students also choose to purchase more credits here to move even faster. **Please note that this is a quick fix and not a solution as inefficient code will lead to fast depletion of credits**

If you are having GPU issues please feel free to make an Ed post or attend an Office Hour and we will help you out.

In [ ]:
# DO NOT MODIFY
torch.manual_seed(42) # For grading consistency
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.__version__)
print(device)

2.1.0+cu118
cpu


In [ ]:
device

device(type='cpu')

In [ ]:
# Grader Cell (1 points)
grader.grade(test_case_id = 'device', answer = str(device))

# **Part 1:** Data Preprocessing and Preparation for Modeling in `PyTorch` (Total: 20 points)

In this homework we will tackle the problem of classifying images.

Specifically, we would be looking at the Modified National Institute of Standards and Technology database ([MNIST](https://en.wikipedia.org/wiki/MNIST_database)) dataset.

<div>
<img src="https://s2.loli.net/2023/03/26/GwFJhNeskzE5Ptx.webp", width="800"/>
</div>

Diagram Reference: [Link](https://en.wikipedia.org/wiki/MNIST_database)

[MNIST](https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html) dataset consists of 70,000 28x28 grayscale images in 10 classes.

There are 60,000 training images and 10,000 test images.

While it is good to have as much training data as possible, in order to avoid having long training time and potentially running out of GPU, we will downsample and train the model with 30,000 training images and 5,000 testing images.


We would be using this dataset to train 3 different models:

1.   Softmax Regression
2.   Feedforward (Fully-Connected) Neural Network
3.   Convolutional Neural Network

And analyze the difference between these models by looking at the test accuracy and loss.

## 1.1 Pytorch Dataset and DataLoader (Total: 10 points)

To keep data loading consistent across different models, PyTorch provides two data primitives: `torch.utils.data.DataLoader` and `torch.utils.data.Dataset` that allow you to use pre-loaded datasets as well as your own data.

`Dataset` stores the samples and their corresponding labels. And `DataLoader` wraps an iterable around the Dataset to enable easy access to the samples. More information and examples of the objects in action can be found **[here](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)**

While Pytorch provides `Dataset` and `Dataloader` for some popular datasets (i.e., one can instantiate the objects and the data is ready to be analyzed in PyTorch Machine Learning pipeline), sometimes we need to define our own custom `Dataset` to process our dataset. The list of available built-in datasets PyTorch gave us can be found **[here](https://pytorch.org/vision/stable/datasets.html)**

<div>
<img src="https://s2.loli.net/2023/03/30/yxbP8gXCroO1Y7c.png", width="800"/>
</div>

Diagram Reference: [Link](https://www.kaggle.com/code/uvxy1234/cifar-10-implementation-with-pytorch)

Dataset class is defined with 3 components:

1.   __init__ : setting up the parameters being used in the class (e.g., `transforms` which corresponds to the transformation being applied)
2.   __len__ : so that len(dataset) returns the size of the dataset.
3.   __getitem__ to support the indexing such that `dataset[i]` can be used to get `i`ith sample (in our case Image, label pair).

To create a custom Dataset object, you would need overide above functions.

Documentation for creating custom dataset can be found [here](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html).

Luckily, as MNIST is a classic image dataset, we do not need to worry about creating custom `Dataset` and `DataLoader` objects since they are all built into the PyTorch when you import that above.

Complete the missing code in the two sections below.

*   **1.1.1 Instantiate Train and Test Dataset.** $\to$ Load and transform data such that it is ready to be passed into the model
*   **1.1.2 Instantiate Train and Test Dataloader.** $\to$ Put the data into the iterable such that the model is able to process data in batch from using `DataLoader`


### 1.1.1 Instantiate Dataset (for train/test dataset) (6 points)

One optional argument in Dataset class is `transform`, where we can apply a transformation to the train dataset before we load them as a dataloader.

For example if we define a `Resize` function in transform, all of our dataset(images) would be resized to specified size passed in as an argument.

---
**TODO**: complete the code to apply following transformations (in order!!) by using `transforms.Compose` and save it to the variable `transform`:

1.   Resize the image to 28 by 28 $\to$ just to make sure they are really have size $28$x$28$
2.   Convert the images to Tensor
3.   Rotate the image using `RandomRotation` with the range of rotation between $(-45°,45°)$
4.   Add random perspective shift using `RandomPerspective` with the distortion of `0.3` to `50%` of the data.
5.   Normalize the Tensor using mean value of `0.1307` and standard deviation value of `0.3081`
---
**NOTE**: We highly recommend looking at [PyTorch documentation](https://pytorch.org/vision/stable/transforms.html) and some [illustrations](https://pytorch.org/vision/stable/auto_examples/transforms/plot_transforms_illustrations.html#sphx-glr-auto-examples-transforms-plot-transforms-illustrations-py) before proceeding with the code

Please review the the recitation notebook for detailed instructions on how to perform these operations.

**FYI**: The values `0.1307` and `0.3081` are mean and standard deviation, respectively, of the MNIST dataset [Ref](https://discuss.pytorch.org/t/normalization-in-the-mnist-example/457/4)

**NOTE**: Please store the distortion of image in the variable `distortion_scale` since it will be used as an input into the autograder.

---
**TODO**:

Similarly, choose the right transforms and apply them to the test dataset. When applying transforms to test set, we don't apply the ones responsible for data augmentation. Which ones of the transforms that we applied on the training data are for data augmentation?

In [ ]:
scale = 28

# # TODO - Define transforms for train set (Remember to store the value of distortion scale in 'distortion_scale' variable)


# TODO END


# TODO - Define transforms for test set


# TODO END


Now that we have defined the tranforms we are going to apply to the dataset, let's instantiate `Dataset` objects for both the training and testing sets using predefined PyTorch Dataset. Refer to the documentation [here](https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html).

In addition, since the whole dataset is pretty large for running the model, we will also be downsampling by reducing the size of the dataset by half.

---
**TODO**:
* Initiate the Dataset object for the training set as `train_dataset`
* Downsample training by getting the train data at even-numbered indices, specified as `train_subset`
* Initiate the Dataset object for the testing set as `test_dataset`
* Downsample `test_dataset` by getting the test data at even-numbered indices, specified as `test_subset`
---
---

**Hint**: Check out [`torch.utils.data.Subset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Subset) for downsampling the dataset.

**Note**: When initiating the `Dataset` object for both training and testing sets, in addition to other arguments please set these arguments as follows:
* `root = './data'`
* `download = True`

In [ ]:
# TODO

# TODO END

**NOTE**: Please do not forget to store the distortion of image in the variable `distortion_scale`.

In [ ]:
#Grader Cell (6 points)
transforms_train = []
for i in range(len(train_dataset.transform.transforms)):
  transforms_train.append(str(train_dataset.transform.transforms[i]))
transforms_test = []
for i in range(len(test_dataset.transform.transforms)):
  transforms_test.append(str(test_dataset.transform.transforms[i]))
grader.grade(test_case_id = 'instantiate_dataset', answer = (transforms_train, transforms_test, len(train_subset), len(test_subset), distortion_scale))

**Hint:** For the above grader cell, part of the criterias we are checking is if the **transform** object is set up correctly.

### 1.1.2 Dataloader - Train / Test (4 points)

Now that we have `train_subset` and `test_subset`, let's create dataloaders using these two datasets.

You can load the dataset into dataloaders using the `DataLoaders` object. Please refer to the documentation [here](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader).

---
**TODO**:
* Initialize the `DataLoader` object for `train_subset` as `train_loader`
* Initialize the `DataLoader` object for `test_subset` as `test_loader`
---

**NOTE**: set the `batch_size` equal to the previously-defined variable `batch`, set `shuffle` to `True`, and `num_workers` as 0.

**SHUFFLE**: Do you think shuffling is necessary on test set?

In [ ]:
batch = 64
# TODO

# TODO END

In [ ]:
#Grader Cell (4 points)
trainLoaderCheck = [getattr(train_loader, "batch_size"),
                    str(type(train_loader)),
                    len(getattr(train_loader, "dataset")),
                    str(getattr(train_loader, "sampler")),
                    getattr(train_loader, "num_workers")]
testLoaderCheck = [getattr(test_loader, "batch_size"),
                    str(type(test_loader)),
                    len(getattr(test_loader, "dataset")),
                    str(getattr(test_loader, "sampler")),
                    getattr(test_loader, "num_workers")]
grader.grade(test_case_id = 'check_dataloader', answer = (trainLoaderCheck, testLoaderCheck))

## 1.2 Summarizing our Dataset (Total: 10 points)

Getting a good sense of the dataset we are going to work on is always the first step you should take when implementing ML applications.

In this section, we will be looking at the distribution of the dataset (e.g., how many instances belong to class with label `0`) and visualize what we are dealing with (i.e., plot out the sample images)

### 1.2.1 Looking at the distribution of labels (6 points)

We can look at the distribution of labels by retrieving the labels of all possible instances of the subset of data pulled from `train_dataset` and `test_dataset` (i.e., `train_subset` and `test_subset`) for the training and testing data, respectively. We defined this in section `1.1.1`.

---
**TODO**
* Train
  * Get the number of unique labels in `train_subset` and save this quantity as the variable `train_num_labels`.
  * Create a `DataLoader` called `train_loader_bar_plot` which takes as arguments the `train_subset`, with the following parameters:
    * `batch_size` = length of the `train_subset`
    * `num_workers = 0`
  * Create a dictionary `train_subset_dict` that contains the labels as keys and the number of images inside the label as values.
* Test
  * Get the number of unique labels in testing dataset and save this quantity as the variable `test_num_labels`.
  * Create a `DataLoader` called `test_loader_bar_plot` which takes as arguments the `test_subset`,  with the following parameters:
    * `batch_size` = length of the `test_subset`
    * `num_workers = 0`
  * Create a dictionary `test_subset_dict` that contains the labels as keys and the number of images inside the label as values.
---
Please refer to **[this](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)** for instantiating the `DataLoader`, which is similar to what we did in section `1.1.2`

In [ ]:
# TODO - TRAIN Data

# Use the original train dataset (with 60,000 images) object to obtain the number of label classes train data


# Creating dictionary for train dataset

# END TODO

print(train_subset_dict)

In [ ]:
# TODO - TEST Data

# Use the original test dataset (with 10,000 images) object to obtain the number of label classes in test data


# Creating dictionary for test dataset

# END TODO
print(test_subset_dict)

In [ ]:
#Grader Cell (6 points)
grader.grade(test_case_id = 'dataset_dict', answer = (train_num_labels, train_subset_dict, test_num_labels, test_subset_dict))

### 1.2.2 Visualize through bar charts (2 points - Manual Grading)

Now we are going to visualize the distribution of labels using bar charts for both training and testing set which we store the distributions in their respective dictionary objects in section 1.2.1.

---
**TODO: You can use either `matplotlib` or `seaborn` for this section**
* Create a barplot for the distributions of training labels using `train_subset_dict` with x-axis label as `Labels` and y-axis label as `Frequency`, and title as `Training set labels and corresponding frequencies`
* Create a barplot for the distributions of testing labels using `test_subset_dict` with x-axis label as `Labels` and y-axis label as `Frequency`, and title as `Testing set labels and corresponding frequencies`
* Add corresponding `count labels` to the barplots (i.e., if label 1 has a
count of 1200, "1200" should appear above the bar corresponding to label 1)
-  To avoid "File Too Large" for submission, please set figure size to (8,6).
---
**NOTE**:

* y-axis $\to$ Number of data points
* x-axis $\to$ The integer value of the labels in the MNIST dataset

In [ ]:
# TODO: Create a barplot showing the distribution of the dataset
# TRAIN Data


In [ ]:
# TODO: Create a barplot showing the distribution of the dataset
# TEST Data


### 1.2.3 Visualize the Training Dataset! (2 points - Manual Grading)

Since everything tends to make more sense when one could literally see it, we now ask you to visualize the images in the `train_dataset` given a fixed set of indices.

---

**TODO**
* loop through the `train_dataset` using `sample_idxs` (the list of indices provided) using a for-loop
* For each iteration of this for-loop:
  * Plot the current image in grayscale from `train_dataset` indexed by current element in `sample_idxs` using matplotlib in a $2x3$ subplot (2 rows, 3 columns)
  * Assign the title of each image to be their respective labels using `plt.title`
  * Hide grid lines and axes tick labels
  * To avoid "File Too Large" for submission, please set overall figure size to (8,4), not individual image.
---
**HINT**: use `enumerate` in for-loop initialization and apply the current loop number to the subplot

**NOTE 1**: `train_dataset` can be indexed by `train_dataset[index]` and you should play around with the output format to get the image and labels



In [ ]:
torch.manual_seed(42)
sample_idxs = [10, 300, 700, 2708, 5035, 8000] # DO NOT MODIFY

# # TODO
# loop through the length of tickers and keep track of index


# END TODO

# **Part 2:** Classification Models (Total: 79)

We now have the data needed to train a multi-class object classifier. We will start simple with a softmax regression classifier as a baseline for our performance, before we move onto more complex neural networks.

In this case, we are looking at the remaining part in the pipeline which were grayed out before as follows:

<div>
<img src="https://s2.loli.net/2023/03/30/ZCBFQvkXuoJpI8K.png", width = "800"/>
</div>

Diagram Reference: [Link](https://www.kaggle.com/code/uvxy1234/cifar-10-implementation-with-pytorch)

## 2.1 Softmax Regression - Baseline (Total: 15 points)

Let's first try solving this problem with a Softmax Regression classifier.

[Softmax Regression](https://medium.com/@tpreethi/softmax-regression-93808c02e6ac) is a multi-class extension of the binary Logistic Regression classifier. Contrary to Logistic Regression which outputs a single number as the class probability, Softmax Regression has as many outputs as the number of classes.

We will define a Softmax Regression model in Pytorch and train it on our training set and evaluate the performance on the test set.

Softmax Regression can be modelled as a function that can take in any number of inputs and give outputs for how likely for a sample is to belong to each class for our classification task.

Similar to a [Sigmoid activation](https://en.wikipedia.org/wiki/Sigmoid_function) in Logistic Regression, a [Softmax Classifier](https://en.wikipedia.org/wiki/Softmax_function) applies the Softmax function on the model output to give class probabilities for all the classes.

**NOTE**:

In PyTorch, when the [Cross Entropy Loss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html), is calculate it applies a softmax function on the output by default, so you don't need to explicitly apply softmax inside your model definition!

<div>
<img src='https://miro.medium.com/v2/resize:fit:4800/format:webp/1*xjIHYMcUlrGsdaxq9FDJxA.png',width='600'/>
</div>

Diagram Reference: [Link](https://medium.com/@lomashbhuva/softmax-regression-a-comprehensive-guide-to-multi-class-classification-491202552135)

### 2.1.1 Softmax Regression Model Architecture (5 points)

We will define our first model in Pytorch. Read up about how to define and use layers in a Pytorch neural network [here](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html).

Our first model is a softmax regression model with the number of outputs equal to the number of classes in the model. Please implement the constructor for the `SoftmaxReg` class (the `__init__` function below) with the definition of the softmax regression model.

**NOTE**: When trying to reshape the features, use the appropriate PyTorch layer instead of tensor reshape/view methods.

In [ ]:
class SoftmaxReg(nn.Module):
    def __init__(self):
        super().__init__()
        # TODO: initialize the neural network layers

        # END TODO

    def forward(self, x):
        # TODO: implement the operations on input data

        # END TODO
        return outputs

Let's print the model summary

In [ ]:
SoftmaxReg()

In [ ]:
#Grader Cell (5 points)
grader.grade(test_case_id = 'softmax_regression_model', answer = str(SoftmaxReg()))

**Notes:** Getting full credit for the above test case doesn't necessarily ensure that your model will perform well on the test dataset as it only evaluates the structure of your neural network.

If you don't achieve complete credit for accuracy and loss on the test dataset, you may need to modify your architecture.

### 2.1.2 Training Softmax Regression Model (6 points)
---
**TODOs:**
1.   Instantiate the Softmax Regression to the variable `softmaxreg`. Make sure to send this to the GPU device -- please consult the [Module 20 slides](https://docs.google.com/presentation/d/1pw9cFO30U_kI47JUZIW9COLAkM8BE8wr/edit?usp=sharing&ouid=115607071793138372214&rtpof=true&sd=true) for a primer on how to do this.

2.   Set the loss criterion as `CrossEntropyLoss` (you can look up the documentation [here](https://pytorch.org/docs/stable/nn.html#loss-functions)). Note that softmax is already built into CrossEntropyLoss so if you use CrossEntropyLoss as the loss criterion, you don't need to add an additional softmax layer.

3.   Fill in the missing parts in the training loop (see `#TODO` comments below)
4.   Save the Training Accuracy for every epoch into the variable `acc_LIST_soft`.
5.   Save the Average Loss for every epoch into the variable `loss_LIST_soft`.
---
We will use `train_loader` built in 1.1.2 to train Softmax Regression model.

The optimizer is set as Adam -- **please do not modify the optimizer**.

Hint: Remember to update the weights correctly by backpropagation, please zero out the gradients by calling `optimizer.zero_grad()` every time you call `backward()`.

**Note: If the loss went up during the training, there is something wrong with the model, so you should check if the model is implemented correctly**

Note: `acc_LIST_soft` and `loss_LIST_soft` should contain data of type float not tensor.

In [ ]:
%%time
# Sending the data to device (CPU or GPU)
# TODO: (1 of 2)
# Step 1: instantiate the Softmax Regression to variable softmaxreg

# Step 2: set the loss criterion as CrossEntropyLoss

# END TODO
optimizer = optim.Adam(softmaxreg.parameters(), lr=1e-4) #lr - learning step
epoch = 10

loss_LIST_soft = []
acc_LIST_soft = []

# Train the Softmax Regression
for epoch in range(epoch):
  running_loss = 0.0
  correct = 0
  total = 0
  for inputs, labels in train_loader:
      labels = labels.type(torch.LongTensor) # Cast to Float
      inputs, labels = inputs.to(device), labels.to(device)

      ## TODO (2 of 2)
      # Step 1: Reset the optimizer tensor gradient every mini-batch

      # Step 2: Feed the network the train data

      # Step 3: Get the prediction using argmax

      # Step 4: Find average loss for one mini-batch of inputs

      # Step 5: Do a back propagation

      # Step 6: Update the weight using the gradients from back propagation by learning step

      # Step 7: Get loss and add to accumulated loss for each epoch

      # Step 8: Get number of correct prediction and increment the number of correct and total predictions after this batch
      # Hint: we need to detach the numbers from GPU to CPU, which stores accuracy and loss

  # Step 9: Calculate training accuracy for each epoch (should multiply by 100 to get percentage), store in variable called 'accuracy', and add to acc_LIST_soft

  # Step 10: Get average loss for each epoch and add to loss_LIST_soft

  # END TODO

  # print statistics
  print("The loss for Epoch {} is: {}, Accuracy = {}".format(epoch, round(running_loss/len(train_loader), 4), round(accuracy, 2)))

In [ ]:
# (if applicable)
# In order to pass the AutoGrader, every element in "acc_LIST_soft" should be a float.
# If the elements are of type "Tensor", convert each element into type FLOAT by using .item() or .tolist()
# To check if each element is a Tensor, print out "acc_LIST_soft" and see if there is a Tensor() wrapped around each element.


In [ ]:
# Grader Cell (6 points)
grader.grade(test_case_id = 'soft_train_loss', answer = (acc_LIST_soft, loss_LIST_soft))

### 2.1.3 Plotting Training Accuracy vs Epochs for Softmax Regression (2 points - Manually Graded)

---
**TODO:**

Plot the training accuracy vs epochs.

Chart Specifications:
1. The accuracy should be in the y-axis and epochs in x-axis.
2. Add chart title.
3. Epoch label should start with 1 (for audience interpretability).
4. To avoid "File Too Large" for submission, please set figure size to (5,3).
---

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# TODO

# END TODO


### 2.1.4 Softmax Regression Model Accuracy (2 points)

Calculate the Test Accuracy for the Softmax Regression Model we trained above.

This should be similar to how we calculated training accuracy above.

In [ ]:
total = 0
correct = 0
with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.type(torch.LongTensor) # Cast to Float
        images, labels = images.to(device), labels.to(device)

        # TODO
        # Get the output

        # Get the prediction using argmax

        # Get number of correct prediction and add to correct and total

# Calculate test accuracy for softmax regression (should multiple by 100)
# NOTE: Please convert the tensor to a float using .item() and store it as a float
test_acc_soft =

# TODO END
print('Test Accuracy: ' + str(test_acc_soft))

In [ ]:
#Grader Cell (2 points)
grader.grade(test_case_id = 'softmax_regression_test_acc', answer = test_acc_soft)

## 2.2 Feedforward Neural Networks (Total: 25 points)

<div>
<img src='https://s2.loli.net/2022/11/21/dvqstVUzcQPChD1.png', width='400'/>
</div>

Diagram reference: [Link](https://en.wikipedia.org/wiki/Feedforward_neural_network)

Since softmax regression isn't that great at the classification problem above, we need more representation power in our model. We will now define a feedforward neural network.

Complete the *FNN* class below to define a feedforward neural network with **only 1 hidden layers with ```out_features``` of 256**. Note that the last layer must have the same number of classes as the output size!

When implementing this model, please think about what activation function would be most appropriate for each layer in this classifier.

### 2.2.1 Feedforward Neural Network Model Architecture (10 points)

In [ ]:
class FNN(nn.Module):
    def __init__(self):
        super().__init__()
        # TODO: initialize the neural network layers


        # END TODO

    def forward(self, x):
        # TODO: implement the operations on input data
        # Hint: think of the neural network architecture for FNN


        # END TODO
        return outputs

Let's print the model summary

In [ ]:
FNN()

In [ ]:
#Grader Cell (10 points)
grader.grade(test_case_id = 'feedforward_nn_model', answer = str(FNN()))

**Notes:** Getting full credit for the above test case doesn't necessarily ensure that your model will perform well on the test dataset as it only evaluates the structure of your neural network.

If you don't achieve complete credit for accuracy and loss on the test dataset, you may need to modify your architecture such as increasing the number of hidden layers or the number of neurons in each layer.

### 2.2.2 Training FNN Model (11 points)
---
TODOs
1.   Instantiate the FNN to the variable `fnn` (make sure to send this to the device env)
2.   Set the loss criterion to be `CrossEntropyLoss` (you can look up the documentation [here](https://pytorch.org/docs/stable/nn.html#loss-functions))
3.   Fill in the missing parts in the training loop below.
4.   Save the Training Accuracy for every epoch into `acc_LIST_FNN`
5.   Save the Average Loss for every epoch into `loss_LIST_FNN`
---
The optimizer is set as Adam -- **please do not modify the optimizer**.

Hint: Remember to update the weights correctly by backpropagation, please zero out the gradients by calling `optimizer.zero_grad()` every time you call `backward()`.

**Note: If the loss went up during the training, there is something wrong with the model, so you should check if the model is implemented correctly**

The following code bloack should take around 4-5 minutes to complete.

Note: `acc_LIST_FNN` and `loss_LIST_FNN` should contain data of type float not tensor.

In [ ]:
%%time
# Sending the data to device (CPU or GPU)
# TODO (1 of 2)
# Step 1: instantiate the FNN model to variable fnn

# Step 2: set the loss criterion as CrossEntropyLoss

# END TODO
optimizer = optim.Adam(fnn.parameters(), lr=1e-4) #lr - learning step
epoch = 10

acc_LIST_FNN = []
loss_LIST_FNN = []

# Train the FNN
for epoch in range(epoch):
  running_loss = 0.0
  correct = 0
  total = 0
  for inputs, labels in train_loader:
      labels = labels.type(torch.LongTensor) # Cast to Long
      inputs, labels = inputs.to(device), labels.to(device)

      ## TODO (2 of 2)
      # Step 1: Reset the optimizer tensor gradient every mini-batch

      # Step 2: Feed the network the train data

      # Step 3: Get the prediction using argmax

      # Step 4: Find average loss for one mini-batch of inputs

      # Step 5: Do a back propagation

      # Step 6: Update the weight using the gradients from back propagation by learning step

      # Step 7: Get loss and add to accumulated loss for each epoch

      # Step 8: Get number of correct prediction and increment the number of correct and total predictions after this batch
      # Hint: we need to detach the numbers from GPU to CPU, which stores accuracy and loss

  # Step 9: Calculate training accuracy for each epoch (should multiply by 100 to get percentage), store in variable called 'accuracy', and add to acc_LIST_FNN

  # Step 10: Get average loss for each epoch and add to loss_LIST_FNN

  # END TODO

  # print statistics
  print("The loss for Epoch {} is: {}, Accuracy = {}".format(epoch, running_loss/len(train_loader), accuracy))


In [ ]:
# (if applicable)
# In order to pass the AutoGrader, every element in "acc_LIST_FNN" should be a float.
# If the elements are of type "Tensor", convert each element into type FLOAT by using .item() or .tolist()
# To check if each element is a Tensor, print out "acc_LIST_FNN" and see if there is a Tensor() wrapped around each element.


In [ ]:
#Grader Cell (11 points)
grader.grade(test_case_id = 'fnn_train_loss', answer = (acc_LIST_FNN, loss_LIST_FNN))

### 2.2.3 Plotting Training Accuracy vs Epochs FNN (2 points - Manually Graded)

---
**TODO:**

Plot the training accuracy vs epochs.

Chart Specifications:
1. The accuracy should be in the y-axis and epochs in x-axis.
2. Add chart title.
3. Epoch label should start with 1 (for audience interpretability).
4. To avoid "File Too Large" for submission, please set figure size to (5,3).
---

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# TODO

# END TODO


### 2.2.4 FNN Model Accuracy(2 points)

Calculate the Test Accuracy for the FNN Model we trained above (the technique for doing this is the same as computing the test accuracy for the softmax regression classifier above).

In [ ]:
total = 0
correct = 0
with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.type(torch.LongTensor) # Cast to Float
        images, labels = images.to(device), labels.to(device)

        # TODO
        # Get the output

        # Get the prediction using argmax

        # Get number of correct prediction and add to correct and total


# Calculate test accuracy for FNN (should multiple by 100)
# NOTE: Please convert the tensor to a float using .item() and store it as a float
test_acc_FNN =
# TODO END

print('Test Accuracy: ' + str(test_acc_FNN))

In [ ]:
#Grader Cell (2 points)
grader.grade(test_case_id = 'feedforward_nn_acc', answer = (test_acc_FNN, loss_LIST_FNN))

##2.3 "Convoluted" Convolutional Neural Networks (Total: 29 points)
So, what are CNNs?

Convolutional Neural Networks are very similar to Feedforward Neural Networks from the previous section: they are made up of neurons that have learnable weights and biases. Each neuron receives some inputs, performs a dot product and optionally follows it with a non-linearity. The whole network still expresses a single differentiable score function: from the raw image pixels on one end to class scores at the other.

So what changes? ConvNet architectures make the explicit assumption that the inputs are images, which allows us to encode certain properties into the architecture. These then make the forward function more efficient to implement and vastly reduce the amount of parameters in the network.

If you wanna know more about how CNNs function and see some cool visualizations, we would highly recommend this [page](https://cs231n.github.io/convolutional-networks/)

<div>
<img src='https://s2.loli.net/2022/11/21/L6pUz2chXWRGn31.png', width='800'>
</div>

Diagram Reference: [Link](https://www.analyticsvidhya.com/blog/2020/10/what-is-the-convolutional-neural-network-architecture/)

We will define the architecture for the CNN we will be using. The components of a CNNs are as follows:

*   Convolutional Layers
*   Pooling Layers
*   Linear Layers
*   Activation Functions

Define a CNN model with Pytorch that contains one or more blocks, where **each block** consists of a **convolutional layer** followed by an **activation function** and a **max pool**. (For this assignment, we discourage adding three or more layers.)

Then, **flatten** the output from the convolutional layers, and pass it through one or more fully connected or 'dense' layers and activation functions after all but the last layer. Note that the output shape from the last layer must be the same as the number of classes.

### 2.3.0 Calculating Output Dimensions of Convolution and Pooling Layers (3 Points)

Before we start building the architecture, it's important to understand the dimensions of the feature maps produced in each of the convolution and pooling layers. We want to keep track of this information as once we get to the fully-connected layers, we need to specify the number of input parameters.

Let's first build a function that will help us calculate the dimensions based on the input parameters.


Since we are working with squared image, we will build just one function that can be used to calculate both the H (height) and W (width) of the feature map.

Below is the function used to calculate the output dimension of the feature map:

$$
O = \left\lfloor \frac{I - f + 2p}{s} \right\rfloor + 1
$$

Where:
- `I`: Input size (height or width)
- `f`: Filter (kernel) size
- `p`: Padding applied on both sides of the input
- `s`: Stride (default = 1)
- $\lfloor . \rfloor$: Floor operation, which rounds down to the nearest integer.


The formula computes how many times the filter fits into the padded input space along that dimension.

---

---
**TODO:**

Build the **feature_map_dim** function, which takes in the following four parameters and return the dimension of the output feature map.

- **input_dim**: height/width of the input image

- **kernel_size**: size of the convolving kernel (aka. filter)

- **padding**: padding added to all four sides of the input in order to preserve the edges/borders

- **stride**: Stride of the convolution, or how many shifts per kernel
  
ROUND DOWN if the result if a fraction.

---

Notice there is another parameter, "dilation", which is the spacing between kernel elements. We have not covered this in class; simply set it to 1.


In [ ]:
import math

def feature_map_dim(input_dim, padding, kernel_size, stride):
  '''
  The function takes in the following four parameters and return the dimension of the output feature map.

  input_dim: height/width of the input image
  kernel_size: size of the convolving kernel (aka. filter)
  padding: padding added to all four sides of the input in order to preserve the edges/borders
  stride: Stride of the convolution, or how many shifts per kernel

  ROUND DOWN if the result if a fraction.

  Notice there is another parameter, "dilation", which is the spacing between kernel elements. We have not covered this in class; simply set it to 1
  '''
  ## TODO:


  ## END TODO

In [ ]:
#Grader Cell (3 points)
grader.grade(test_case_id = 'test_func_feature_map_dim', answer = getsource(feature_map_dim))

### 2.3.1 Convolutional Neural Network Model Architecture (12 points)

---
**TODOs:**

`def __init__(self)`::
1.  Initialize 1 - 3 `convolutional blocks` (consists of a convolution layer, an activation function, and a Pooling layer)

2. `Flatten` the output of the convolution to a tensor

3. Initialize 1-3 `fully-connected layers`

`def forward(self, x)`:
4. Build the `forward` function/path with the layers initialized in `__init__` function.

---

- **Note 1:** While parameters of convolution layers tend to vary, it is possible to use the same pooling layer and activation function in each block. If you intend to use the same parameters for these two elements, then only 1 pooling layer and 1 activation function needs to be initialized.
- **Note 2** : When trying to reshape the features, use the appropriate PyTorch layer instead of tensor reshape/view methods.
- **Hint 1**: Use the `feature_map_dim` function, and the input_dim and output_dim comments to help you keep track of the input/output dimensions of each layer
- **Hint 2**: The parameters you've calculated is particularly useful for the input dimensions for the linear layer in the first fully-connected layer.
- **Hint 3**: Each input is a grey-scaled (1 channel) 28 x 28 image.
- **Hint 4**: The final fully-connected layer's output dimension should be the same as the number of classes in our dataset


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # TODO

        # Step 1: Initialize 1 - 3 convolution blocks (consists of a convolution layer, an activation function, a MaxPooling layer)

        ## you are encouraged to use the following comments to keep track of the output dimensions
        # input dim = __ channels, with image size __ x __
        # output dim = __ channels, with image size __ x __


        # Convolution Block 1

        # Convolution Block 2 ... Feel free to add 1 - 2 more convolution blocks

        # Step 2: Flatten the 2D image into a 1D tensor


        # Step 3: Initialize 1-3 fully-connected layers


        # END TODO

    def forward(self, x):
        # TODO

        # Step 1. Pass the images (x) through convolution block 1 and (block 2, 3 if you have built them)


        # Step 2. Flatten the image


        # Step 3. Pass the output through the fully-connected layers (remember to include activation function(s))


        # END TODO
        return outputs

Let's print out the model summary

In [ ]:
CNN()

In [ ]:
#Grader Cell (12 points)
grader.grade(test_case_id = 'cnn_model', answer = str(CNN()))

### 2.3.2 Training CNN Model (10 Points)

---
**TODOs**
1.   Instantiate the CNN to the variable `cnn` (make sure to send this to the device env)
2.   Set the criterion as a CrossEntropyLoss (you can look up the documentation [here](https://pytorch.org/docs/stable/nn.html#loss-functions))
3.   Fill in the missing parts in the training loop.
4.   Calculate training accuracy for each epoch (should multiply by 100 to get percentage), store in variable called `accuracy`.
5.   Save the Training Accuracy for every epoch into `acc_LIST_CNN`
6.   Save the Average Loss for every epoch into `loss_LIST_CNN`

**Please do not modify the following set ups: **
Optimizer: The optimizer is set as "Adam".
Epoch: The epoch is set to "10".

---

**Hint:** Remember to update the weights correctly by backpropagation, please zero out the gradients by calling `optimizer.zero_grad()` every time you call `backward()`.  Please review the `Recitation 10` notebook for detailed instructions on how to perform these operations.

**Note 1**: If the loss went up during the training, there is something wrong with the model, so you should check if the model is implemented correctly.

**Note 2**: Typical training time takes between 5 - 10 mins.

**Note 3**: `acc_LIST_CNN` and `loss_LIST_CNN` should contain data of type float not tensor.

In [ ]:
%%time
# Sending the data to device (CPU or GPU)
# TODO (1 of 2)
# Step 1: instantiate the CNN model to variable cnn

# Step 2: set the loss criterion as CrossEntropyLoss

# END TODO
optimizer = optim.Adam(cnn.parameters(), lr=1e-4) #lr - learning step
epoch = 10

acc_LIST_CNN = []
loss_LIST_CNN = []

# Train the CNN
for epoch in range(epoch):
  running_loss = 0.0
  correct = 0
  total = 0
  for inputs, labels in train_loader:
      labels = labels.type(torch.LongTensor) # Cast to Float
      inputs, labels = inputs.to(device), labels.to(device)
      ## TODO (2 of 2)
      # Step 1: Reset the optimizer tensor gradient every mini-batch

      # Step 2: Feed the network the train data

      # Step 3: Get the prediction using argmax

      # Step 4: Find average loss for one mini-batch of inputs

      # Step 5: Do a back propagation

      # Step 6: Update the weight using the gradients from back propagation by learning step

      # Step 7: Get loss and add to accumulated loss for each epoch

      # Step 8: Get number of correct prediction and increment the number of correct and total predictions after this batch
      # Hint: we need to detach the numbers from GPU to CPU, which stores accuracy and loss

  # Step 9: Calculate training accuracy for each epoch (should multiply by 100 to get percentage), store in variable called 'accuracy', and add to acc_LIST_CNN

  # Step 10: Get average loss for each epoch and add to loss_LIST_CNN

  # END TODO

  # print statistics
  print("The loss for Epoch {} is: {}, Accuracy = {}".format(epoch, running_loss/len(train_loader), accuracy))


In [ ]:
# (if applicable)
# In order to pass the AutoGrader, every element in "acc_LIST_CNN" should be a float.
# If the elements are of type "Tensor", convert each element into type FLOAT by using .item() or .tolist()
# To check if each element is a Tensor, print out "acc_LIST_CNN" and see if there is a Tensor() wrapped around each element.


In [ ]:
#Grader Cell (10 points)
grader.grade(test_case_id = 'cnn_train_loss', answer = (acc_LIST_CNN, loss_LIST_CNN))

### 2.3.3 Plotting Training Accuracy vs Epochs CNN (2 points - Manually Graded)

---
**TODO:**

Plot the training accuracy vs epochs.

Chart Specifications:
1. The accuracy should be in the y-axis and epochs in x-axis.
2. Add chart title.
3. Epoch label should start with 1 (for audience interpretability).
4. To avoid "File Too Large" for submission, please set figure size to (5,3).
---

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# TODO


# END TODO

### 2.3.4 CNN Model Test Accuracy (2 points)

---
**TODO:**
Calculate the Test Accuracy for the CNN Model we trained above. Save it to the variable named `test_acc_CNN`. The technique for doing this is the same as computing the test accuracy for the softmax regression and FNN classifiers above.

---

In [ ]:
# TODO
total = 0
correct = 0
with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.type(torch.LongTensor) # Cast to Float
        images, labels = images.to(device), labels.to(device)
        # TODO
        # Get the output

        # Get the prediction using argmax

        # Get number of correct prediction and add to correct and total


# Calculate test accuracy for CNN (should multiple by 100)
# NOTE: Please convert the tensor to a float using .item() and store it as a float
test_acc_CNN =
# TODO END

print(f'Test Accuracy: ' + str(test_acc_CNN))

In [ ]:
#Grader Cell (2 points)
grader.grade(test_case_id = 'cnn_test_acc', answer = (test_acc_CNN))

## 2.4. Reflection (2 point - Manually Graded)



Let's compare the model performance:

From the test accuracies, we can see that FNN works better than Softmax Regression, and CNN works even better than FNN.

In [ ]:
# Simply run this cell, please do not modify
print(f'Test Accuracy for Softmax Regression: ' + str(test_acc_soft))
print(f'Test Accuracy for FNN: ' + str(test_acc_FNN))
print(f'Test Accuracy for CNN: ' + str(test_acc_CNN))

---
**TODO:**

Now that you've build an image classification model on a relatively simple dataset, could you think of a real-world practical/complex application/use-case where digit recognition is useful?

---

In [ ]:
# (Manual Grading - 2 point)

# Type your response in the following cell as a comment




## 2.5 Confusion Matrix (Total: 8 points)

We want to give you some insight into how you can further analyze the performance of the classification model you have trained.

Upto now we have only used accuracy as a measure of performance. Although accuracy is simple and widely used measure, there are drawbacks to using accuracy.

Assume that we are trying to train a model to detect a rare disease from CT images of patients. For simplicity, assume that this rare disease has an incidence rate of 1%.

In this situation, if we train a degenerative model which always predicts that the patient does not have the disease, the accuracy is very high, namely 99%! Obviously in this case, accuracy is not a good measure of performance of the model.

This example serves to motivate confusion matrices and related metrics.

A confusion matrix is defined to be $C \times C$ 2D matrix, where $C$ is number of distinct labels in the dataset. In this matrix, the rows represent the actual labels and the columns represent the predicted labels.

The $(i, j)$-th entry of the confusion matrix contains the no. of data points where the actual label is $i$ but the model predicted $j$.

<div>
<img src='https://s2.loli.net/2023/04/02/7T1e5dtYGihvrRN.png', width='600'>
</div>

The provided image is an example of a confusion matrix for binary classification (C = 2), but you can easily imagine this scaling up to C > 2.

From this confusion matrix we can get all sorts of useful metrics such as False Positive Rate (FPR), False Negative Rate (FNR), Recall, Precision, etc... If you are interested in learning more about confusion matrix and related metrics [this wikipedia page](https://en.wikipedia.org/wiki/Confusion_matrix#cite_note-11) is a good place to look at.







### 2.5.1 Create a confusion matrix (4 points)

---

**TODO:**
1. For the test dataset using the CNN model you have trained above, create a confusion matrix.
2. Save the confusion matrix as a pandas dataframe called `confusion_matrix_df`.

---
**Hint:** Look at what you've done in 2.3.4, and see if what you have done can be re-used.

Here's the documentation for Scikit-learn's Confusion Matrix method:

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

**NOTE**: Remember to always use a trained model to construct your confusion matrix. If you reinitialize your model or restart your notebook without retraining your model, you will get incorrect values for confusion matrix.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

# TODO

def cm_generator(test_loader):

  # The goal is to obtain two lists of prediction and actual labels.
  # Then, using these two lists, create a confusion matrix dataframe


# END TODO


In [ ]:
#Grader Cell (4 points)
import numpy as np
grader.grade('check_confusion_matrix', (str(type(confusion_matrix_df)),
                                        confusion_matrix_df.shape,
                                        confusion_matrix_df.to_numpy().sum(),
                                        np.trace(confusion_matrix_df.to_numpy())
                                        ))

### 2.5.2 Visualizing Confusion Matrix (4 points - Manual Grading)

---
**TODOs:**

Implement the code to visualize the confusion matrix you have created above. This section will be manually graded

Suggested library is using Seaborn heatmap.

Chart Specifications:
1. Each cell is labled/annotated with associated values. Take a look at the parameter "annot".
2. Make sure all annotated values are in integers, and not shown in scientific notations. Use the parameter `fmt = "g"`.
3. Add chart title, and axis name label ("Actual" and "Predicted". Check out which axis is associated with either label.
4. Set map color to Green-Blue (GnBu)
5. To avoid "File Too Large" for submission, please set figure size to (8,4).
---
Resource: [Seaborn Heatmap documentation](https://seaborn.pydata.org/generated/seaborn.heatmap.html )

In [ ]:

# TODO


# END TODO


# Homework Submission

Good job! You have finished the homework :) The submission instructions are as follows:

* **Double check** that you have the correct PennID (all numbers) in the autograder.

* **Triple check** that you have all plots shown in this Colab notebook before submitting (otherwise, your submission will be subjected to a **penalty of -5 points**).

*  Go to the "File" tab at the top left of the Colab UI, click "Download .ipynb" and then "Download .py".  **Rename** these two files to `homework5.ipynb` and `homework5.py` respectively and upload them to Gradescope.
  - <ins>**WAIT UNTIL THE GRADESCOPE AUTOGRADER FINISHES RUNNING!**</ins> If we have to manually upload this for you after the deadline, **we will apply a penalty of -5 points** (this stacks with any other penatlies)

* You must submit your notebook to receive credit. Post any issues with submission on Ed and make sure to keep in mind the late day policy.

* After you submit your code, the teaching staff will manually grade your Colab notebook in order to validate the correctness of your code.